In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
x_data_1=np.load("/content/drive/My Drive/ToMau/x_data.npy")
y_data_1=np.load("/content/drive/My Drive/ToMau/y_data.npy")
x_data_2=np.load("/content/drive/My Drive/ToMau/x_data_2.npy")
y_data_2=np.load("/content/drive/My Drive/ToMau/y_data_2.npy")
x_data=np.concatenate((x_data_1, x_data_2), axis=0)
y_data=np.concatenate((y_data_1, y_data_2), axis=0)
print(x_data.shape)
print(y_data.shape)

(20000, 224, 224, 3)
(20000, 224, 224, 2)


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x_data, y_data, test_size=0.2, random_state=0)

In [4]:
print(x_train.shape)

(16000, 224, 224, 3)


In [0]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.applications import resnet50

In [6]:
resnet=resnet50.ResNet50(weights=None, classes=365)
resnet.summary()

W0825 11:01:34.412349 140088183256960 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
x=resnet.output
model_temp=Model(inputs=resnet.input, outputs=x)

In [0]:
layer_3, layer_7, layer_13, layer_16 = model_temp.get_layer('activation_9').output, model_temp.get_layer('activation_21').output, model_temp.get_layer('activation_39').output, model_temp.get_layer('activation_48').output

In [0]:
fcn1 = Conv2D(filters=2 , kernel_size=1, name='fcn1')(layer_16)

fcn2 = Conv2DTranspose(filters=layer_13.get_shape().as_list()[-1] , kernel_size=4, strides=2, padding='same', name='fcn2')(fcn1)
fcn2_skip_connected = Add(name="fcn2_plus_layer13")([fcn2, layer_13])

fcn3 = Conv2DTranspose(filters=layer_7.get_shape().as_list()[-1], kernel_size=4, strides=2, padding='same', name="fcn3")(fcn2_skip_connected)
fcn3_skip_connected = Add(name="fcn3_plus_layer_7")([fcn3, layer_7])

fcn4 = Conv2DTranspose(filters=layer_3.get_shape().as_list()[-1], kernel_size=4, strides=2, padding='same', name="fcn4")(fcn3_skip_connected)
fcn4_skip_connected = Add(name="fcn4_plus_layer_3")([fcn4, layer_3])

fcn5 = Conv2DTranspose(filters=2, kernel_size=16, strides=(4, 4), padding='same', name="fcn5")(fcn4_skip_connected)
relu255 = ReLU(max_value=255) (fcn5)

In [10]:
model=Model(inputs=resnet.input, outputs=relu255)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
model.compile(loss=tf.keras.losses.MSE, optimizer='rmsprop')

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
filepath="/content/drive/My Drive/ToMau/model.h5"
checkpoint=ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True,
                          save_weights_only=True, mode='min')
from time import time
tensorboard=TensorBoard(log_dir="/content/drive/My Drive/ToMau/logs/{}".format(time()))
callbacks_list=[checkpoint, tensorboard]

In [0]:
model.load_weights(filepath)

In [14]:
model.fit(x_train,y_train,batch_size=64,
          epochs=50,
          verbose=1,
          validation_data=(x_test,y_test),
          callbacks=callbacks_list)

W0825 11:01:41.871028 140088183256960 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 16000 samples, validate on 4000 samples
Epoch 1/50
15936/16000 [============================>.] - ETA: 1s - loss: 3065.9734
Epoch 00001: loss improved from inf to 3066.97870, saving model to /content/drive/My Drive/ToMau/model.h5
16000/16000 [==============================] - 382s 24ms/sample - loss: 3066.9787 - val_loss: 3820.9096
Epoch 2/50
15936/16000 [============================>.] - ETA: 1s - loss: 2697.1011
Epoch 00002: loss improved from 3066.97870 to 2697.67257, saving model to /content/drive/My Drive/ToMau/model.h5
16000/16000 [==============================] - 349s 22ms/sample - loss: 2697.6726 - val_loss: 3771.6011
Epoch 3/50
15936/16000 [============================>.] - ETA: 1s - loss: 2487.1931
Epoch 00003: loss improved from 2697.67257 to 2488.97847, saving model to /content/drive/My Drive/ToMau/model.h5
16000/16000 [==============================] - 350s 22ms/sample - loss: 2488.9785 - val_loss: 3525.1077
Epoch 4/50
15936/16000 [============================>.]

In [0]:
model_json = model.to_json()
with open("/content/drive/My Drive/ToMau/model.json", "w") as json_file:
    json_file.write(model_json)